In [62]:
import pandas as pd
df = pd.read_csv("/content/listings_london.csv")

In [63]:
#!unzip listings.csv.zip

In [64]:
#with open('listings.csv.gz', 'rb') as fd:
    #gzip_fd = gzip.GzipFile(fileobj=fd)
    #df = pd.read_csv(gzip_fd)

In [65]:
df.head(3)

,id,listing_url,scrape_id,last_scraped,name,description,neighborhood_overview,picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms,bathrooms_text,bedrooms,beds,amenities,price,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,11551,https://www.airbnb.com/rooms/11551,20201106040109,2020-11-07,Arty and Bright London Apartment in Zone 2,Unlike most rental apartments out there my fla...,Not even 10 minutes by metro from Victoria Sta...,https://a0.muscache.com/pictures/b7afccf4-18e5...,43039,https://www.airbnb.com/users/show/43039,Adriano,2009-10-03,"London, England, United Kingdom","Hello, I'm a friendly Italian man with a very ...",within an hour,100%,100%,f,https://a0.muscache.com/im/pictures/user/47773...,https://a0.muscache.com/im/pictures/user/47773...,Brixton,3.0,3.0,"['email', 'phone', 'reviews', 'jumio', 'offlin...",t,t,"London, United Kingdom",Lambeth,NaN,51.46225,-0.11732,Entire apartment,Entire home/apt,4,NaN,1 bath,1.0,3.0,"[""Cable TV"", ""Coffee maker"", ""Carbon monoxide ...",$97.00,7,190,7,28,1125,1125,8.4,1125.0,NaN,t,0,0,4,279,2020-11-07,192,7,0,2010-03-21,2020-03-26,91.0,9.0,9.0,10.0,10.0,9.0,9.0,NaN,t,2,2,0,0,1.48
1,13913,https://www.airbnb.com/rooms/13913,20201106040109,2020-11-07,Holiday London DB Room Let-on going,My bright double bedroom with a large window h...,Finsbury Park is a friendly melting pot commun...,https://a0.muscache.com/pictures/miso/Hosting-...,54730,https://www.airbnb.com/users/show/54730,Alina,2009-11-16,"London, England, United Kingdom",I am a Multi-Media Visual Artist and Creative ...,within a day,83%,71%,f,https://a0.muscache.com/im/users/54730/profile...,https://a0.muscache.com/im/users/54730/profile...,LB of Islington,3.0,3.0,"['email', 'phone', 'facebook', 'reviews', 'off...",t,t,"Islington, Greater London, United Kingdom",Islington,NaN,51.56802,-0.11121,Private room in apartment,Private room,2,NaN,1 shared bath,1.0,0.0,"[""Cable TV"", ""Bed linens"", ""Coffee maker"", ""Pa...",$40.00,1,29,1,28,29,29,2.8,29.0,NaN,t,30,60,90,365,2020-11-07,21,2,0,2010-08-18,2020-02-22,97.0,10.0,10.0,10.0,10.0,9.0,9.0,NaN,f,2,1,1,0,0.17
2,15400,https://www.airbnb.com/rooms/15400,20201106040109,2020-11-07,Bright Chelsea Apartment. Chelsea!,Lots of windows and light. St Luke's Gardens ...,It is Chelsea.,https://a0.muscache.com/pictures/428392/462d26...,60302,https://www.airbnb.com/users/show/60302,Philippa,2009-12-05,"Kensington, England, United Kingdom","English, grandmother, I have travelled quite ...",within a day,100%,100%,f,https://a0.muscache.com/im/users/60302/profile...,https://a0.muscache.com/im/users/60302/profile...,Chelsea,1.0,1.0,"['email', 'phone', 'reviews', 'jumio', 'govern...",t,t,"London, United Kingdom",Kensington and Chelsea,NaN,51.48796,-0.16898,Entire apartment,Entire home/apt,2,NaN,1 bath,1.0,1.0,"[""Cable TV"", ""Coffee maker"", ""Dryer"", ""Dishes ...",$75.00,10,50,10,28,50,50,11.2,50.0,NaN,t,0,20,50,150,20

In [66]:
df.shape

(76984, 74)

In [67]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
import joblib
from joblib import dump
import scipy

In [68]:
!pip install category_encoders

In [69]:
import category_encoders as ce
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.model_selection import train_test_split


In [70]:
df['bathrooms'].isna().sum()

76984

In [71]:
df['bathrooms'] = df['bathrooms'].fillna(0)

In [72]:
df['bathrooms'].isna().sum()

0

In [73]:
df['bathrooms'].unique()

array([0.])

In [74]:
df['bathrooms'] = df['bathrooms'].replace('t', 0)

In [75]:
df['bathrooms'].unique()

array([0.])

In [76]:
df['bathrooms'] = df['bathrooms'].astype('float')

In [77]:
df['bathrooms'].dtypes

dtype('float64')

In [78]:
df['number_of_reviews'].unique()

array([192,  21,  89,  42,   0, 129,   6,  79, 528,  52,  55, 137, 366,
        36,  10,  84,  71,   2, 167, 197, 107,  35,  11,   1, 102, 373,
        66, 121,  54,   7,  39, 100,  25, 451, 187, 128,  29,  74,  56,
       157,  65, 142,  13, 134,  90,   8, 213, 190,  61,  30,  82,   5,
       415,  97, 173,  86,  46,   9,  69, 342,  37, 212, 181,  23,  41,
       136,  70, 297,  33,  78,  91,  34,  64, 124,  73, 109,   4,  27,
       337, 473, 478, 407,   3,  22,  58, 229, 202,  19, 123,  57,  53,
       103,  43,  98,  45, 161,  85, 175,  18, 147, 149,  17, 180,  51,
        15, 172, 152, 163, 220, 316,  47,  50,  63,  16, 480, 210, 201,
       209, 135, 282,  40, 380, 244, 221, 484,  60, 119, 189,  94, 106,
       139, 274, 110,  20,  62, 108, 223,  24, 343,  32, 159, 430,  44,
       122,  77,  95, 218,  96, 336,  14,  88, 241,  87,  83, 301,  76,
       390,  92,  26, 266, 166, 170, 105, 230, 126, 169, 429, 340,  28,
       160,  48, 214,  12, 257, 146,  68,  99, 168, 582,  75, 25

In [79]:
df['number_of_reviews'] = df['number_of_reviews'].fillna(0)

In [80]:
df['number_of_reviews'] = df['number_of_reviews'].astype('int')

In [81]:
df['number_of_reviews'].dtypes

dtype('int64')

In [82]:
df['price'] = df['price'].fillna(0)

In [83]:
def clean_currency(x):
    """ If the value is a string, then remove currency symbol and delimiters
    otherwise, the value is numeric and can be converted
    """
    if isinstance(x, str):
        return(x.replace('$', '').replace(',', ''))
    return(x)

In [84]:
df['price'] = df['price'].apply(clean_currency).astype('float')

In [85]:
df['price'].dtypes

dtype('float64')

In [86]:

df['host_response_rate'] = df['host_response_rate'].fillna(0)

In [87]:
df['host_response_rate'].unique()

array(['100%', '83%', '67%', 0, '0%', '50%', '57%', '70%', '80%', '90%',
       '75%', '96%', '73%', '89%', '44%', '40%', '71%', '60%', '43%',
       '33%', '85%', '92%', '86%', '25%', '17%', '30%', '22%', '98%',
       '88%', '56%', '94%', '79%', '36%', '95%', '87%', '63%', '97%',
       '93%', '29%', '64%', '91%', '72%', '82%', '20%', '81%', '78%',
       '8%', '38%', '61%', '99%', '18%', '14%', '10%', '68%', '76%',
       '53%', '59%', '9%', '77%', '84%', '23%', '62%', '45%', '6%', '13%',
       '69%', '74%', '46%', '65%', '58%', '41%', '2%', '55%', '4%', '54%',
       '31%', '11%', '7%', '47%'], dtype=object)

In [88]:
df['host_response_rate'] = df['host_response_rate'].apply(lambda x: float(x.strip('%'))/100 if isinstance(x, str) else x )

In [89]:
df['description_len'] = df.description.astype(str).apply(lambda x: len(x))

In [90]:
var_drop = ['id',	'listing_url',	'scrape_id', 	'last_scraped',	'name', 'description',	'neighborhood_overview',	'picture_url','host_id', 'host_url',	'host_name',
            'host_since',	'host_location',	'host_about',	'host_response_time',	'host_acceptance_rate',	'host_is_superhost',
            'host_thumbnail_url',	'host_picture_url',	'host_neighbourhood',	'host_listings_count',	'host_total_listings_count',
            'host_verifications',	'host_has_profile_pic',	'host_identity_verified',	'neighbourhood',	'neighbourhood_cleansed',	'neighbourhood_group_cleansed',
            'latitude',	'longitude',	'room_type',	'bathrooms_text',	'amenities','minimum_nights',	'maximum_nights',
            'minimum_minimum_nights',	'maximum_minimum_nights',	'minimum_maximum_nights',	'maximum_maximum_nights',	'minimum_nights_avg_ntm',
            'maximum_nights_avg_ntm',	'calendar_updated',	'has_availability',	'availability_30',	'availability_60',	'availability_90',	'availability_365',
            'calendar_last_scraped',	'number_of_reviews_ltm',	'number_of_reviews_l30d',	'first_review',	'last_review',	'review_scores_rating',
            	'review_scores_accuracy',	'review_scores_cleanliness',	'review_scores_checkin',	'review_scores_communication',	'review_scores_location',
            	'review_scores_value',	'license',	'instant_bookable',	'calculated_host_listings_count',	'calculated_host_listings_count_entire_homes',
            	'calculated_host_listings_count_private_rooms',	'calculated_host_listings_count_shared_rooms',	'reviews_per_month']
            	


In [91]:
def wrangle(X):
    """Wrangle everything in the same way"""
    
    # Prevent SettingWithCopyWarning
    X = X.copy()

    # Drop the columns with 0 feature importance
    more_variance = []

    X = X.drop(columns=var_drop)

    

    # Engineer features!
    
    # return the wrangled dataframe
    return X

df2 = wrangle(df)

In [92]:
for col in df2.columns:
  print(col)
print(df2.shape)

host_response_rate
property_type
accommodates
bathrooms
bedrooms
beds
price
number_of_reviews
description_len
(76984, 9)


In [93]:
df2.head()

,host_response_rate,property_type,accommodates,bathrooms,bedrooms,beds,price,number_of_reviews,description_len
0,1.00,Entire apartment,4,0.0,1.0,3.0,97.0,192,1000
1,0.83,Private room in apartment,2,0.0,1.0,0.0,40.0,21,1000
2,1.00,Entire apartment,2,0.0,1.0,1.0,75.0,89,1000
3,0.67,Entire apartment,6,0.0,3.0,3.0,307.0,42,1000
4,0.00,Private room in bed and breakfast,2,0.0,1.0,1.0,150.0,0,777


In [94]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76984 entries, 0 to 76983
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   host_response_rate  76984 non-null  float64
 1   property_type       76984 non-null  object 
 2   accommodates        76984 non-null  int64  
 3   bathrooms           76984 non-null  float64
 4   bedrooms            72238 non-null  float64
 5   beds                75904 non-null  float64
 6   price               76984 non-null  float64
 7   number_of_reviews   76984 non-null  int64  
 8   description_len     76984 non-null  int64  
dtypes: float64(5), int64(3), object(1)
memory usage: 5.3+ MB


In [95]:
x = df2.drop(columns='price')
y = df2['price']
x3 = x[[ 'bathrooms', 'bedrooms', 'beds', 'property_type',
        'number_of_reviews', 
        'description_len', 'accommodates']].copy()


In [96]:
x3.head()

,bathrooms,bedrooms,beds,property_type,number_of_reviews,description_len,accommodates
0,0.0,1.0,3.0,Entire apartment,192,1000,4
1,0.0,1.0,0.0,Private room in apartment,21,1000,2
2,0.0,1.0,1.0,Entire apartment,89,1000,2
3,0.0,3.0,3.0,Entire apartment,42,1000,6
4,0.0,1.0,1.0,Private room in bed and breakfast,0,777,2


In [97]:
numeric_features = ['accommodates', 'bathrooms', 'bedrooms', 'beds', 'number_of_reviews',
                     'description_len',]
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

In [98]:
categorical_features = ['property_type']
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('ordinal', OrdinalEncoder())])

In [99]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

X_train, X_test, y_train, y_test = train_test_split(x3, y, test_size=0.2)

X_tra = preprocessor.fit_transform(X_train)

X_tes = preprocessor.fit_transform(X_test)


In [100]:
import category_encoders as ce
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from xgboost import XGBRegressor
import matplotlib.pyplot as plt
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression


In [101]:
reg = LinearRegression(n_jobs=-1)

reg.fit(X_tra, y_train)

y_pred = reg.predict(X_tes)


In [102]:
type(y_test)

pandas.core.series.Series

In [103]:
type(y_pred)

numpy.ndarray

In [104]:
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import r2_score as r2

In [105]:
mae(y_test, y_pred)

60.39819340350519

In [106]:
r2(y_test, y_pred)

0.0408213336952894

In [107]:
from joblib import dump
dump(reg, 'pickle_reg2.joblib', compress=True)

['pickle_reg2.joblib']

In [108]:
dump(preprocessor, 'preprocessor2.pkl')

['preprocessor2.pkl']

In [109]:
order = OrdinalEncoder(categories=categorical_features)

In [110]:
order.categories

pipe3 = make_pipeline(
    order,
    LinearRegression()
)

dump(pipe3, 'pipe4.joblib', compress=True)

['pipe4.joblib']